Default Switch からVMに接続し InternalSwitch に固定IPアドレスを設定する

# 定義

In [ ]:
$vmName           = 'Test-Win11'
$internalSwitch   = 'InternalSwitch'
$vmInternalIP     = '192.168.128.100'
$internalSwitchIP = '192.168.128.254'
$prefixLength     = 24
$cred = New-Object System.Management.Automation.PSCredential (
    'sakaztk',
    (ConvertTo-SecureString -AsPlainText -Force -String 'Passw0rd!')
)

# 内部スイッチが無ければ作る

In [ ]:
if (Get-VMSwitch -Name $internalSwitch -ErrorAction SilentlyContinue) {'OK'}
else {
    New-VMSwitch -Name $internalSwitch -SwitchType Internal -Verbose
    New-NetIPAddress -InterfaceAlias "vEthernet ($internalSwitch)" -IPAddress $internalSwitchIP -PrefixLength $prefixLength
}

# VMにスイッチを接続

In [ ]:
if (-not(Get-VMNetworkAdapter -VMName $vmName | Where-Object { $_.SwitchName -eq $internalSwitch })) {
    Add-VMNetworkAdapter -VMName $vmName -SwitchName $internalSwitch -Verbose
}
Get-VMNetworkAdapter -VMName $vmName

# Default Switch のIPアドレスを取得してCimSessionを作成

## IPアドレス取得

In [ ]:
$dsIpAddress = (Get-VMNetworkAdapter -VMName $vmName | Where-Object {$_.SwitchName -eq 'Default Switch'}).IPAddresses |
  Where-Object {($_ -match '^\d{1,3}(\.\d{1,3}){3}$') -and ($_ -notlike '169.254*') -and ($_ -ne '0.0.0.0')}
$dsIpAddress

## WinRM設定

In [ ]:
$winrm = Get-Service WinRM
if ($winrm.Status -eq 'Stopped' -and $winrm.StartType -eq 'Manual') {
    winrm quickconfig -force
} 

## TrustedHosts設定

In [ ]:
$current = (Get-Item WSMan:\localhost\Client\TrustedHosts).Value
$currentList = if ($current) { $current -split ',' | ForEach-Object { $_.Trim() } } else { @() }
if ($currentList -notcontains $dsIpAddress -and $current -ne '*') {
    $newValue = @($currentList) + @($dsIpAddress) -join ','
    Set-Item WSMan:\localhost\Client\TrustedHosts -Value $newValue -Force -Verbose
}
Get-Item WSMan:\localhost\Client\TrustedHosts

In [ ]:
$cimSession = New-CimSession -ComputerName $dsIpAddress -Credential $cred -Verbose
$cimSession

# 固定IPアドレスを設定
リンクローカルアドレスになっているものにIPアドレスを設定

In [ ]:
Get-NetIPAddress -CimSession $cimSession -AddressFamily IPv4 -IPAddress '169.254.*' |
  New-NetIPAddress -IPAddress $vmInternalIP -PrefixLength $prefixLength -Verbose

# ネットワークプロファイルをプライベートにする

In [ ]:
Get-NetIPAddress -CimSession $cimSession -IPAddress $vmInternalIP |
  Set-NetConnectionProfile -NetworkCategory Private -Verbose -PassThru

# 後処理

In [ ]:
Remove-CimSession $cimSession

## TrustedHosts削除

In [ ]:
$current = (Get-Item WSMan:\localhost\Client\TrustedHosts).Value
if ($current -and $current -ne '*' -and $current -ne '') {
    $hostList = $current -split "," | ForEach-Object { $_.Trim() }
    if ($hostList -contains $dsIpAddress) {
        $updatedList = ($hostList | Where-Object { $_ -ne $dsIpAddress }) -join ","
        Set-Item WSMan:\localhost\Client\TrustedHosts -Value $updatedList -Force
    }
}
Get-Item WSMan:\localhost\Client\TrustedHosts